In [1]:
import numpy as np
import pandas as pd
import shutil

import os,sys
from datetime import datetime

pd.set_option('display.max_columns', None)

In [2]:
df_warehouse=pd.read_excel(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\WAREHOUSE_DATA_FILE.xlsx")

In [3]:
df_warehouse.to_csv("warehouse_data.csv",index=False)

In [4]:
warehouse_Data = pd.read_csv("C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_data.csv")

In [5]:
assest_initiation_invent_sb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_SB.CSV")
assest_initiation_invent_nb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_NB.CSV")

In [7]:
assest_initiation_invent_sb.count()

MeterNumber                                  1012391
MFG_SERIAL_NUM                               1012391
UDC_ID                                       1012131
NETWORK_ID                                   1012131
InstallationNumber                            974272
Status                                       1012391
INSERT_TIME                                  1012391
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1012391
dtype: int64


# changes in dataset like :- UPPER, SPACE REMOVAL , DATATYPE

In [9]:
# Convert all columns to string data type

warehouse_Data = warehouse_Data.astype(str)
assest_initiation_invent_nb = assest_initiation_invent_nb.astype(str)
assest_initiation_invent_sb = assest_initiation_invent_sb.astype(str)



# change column name into upper string

warehouse_Data.columns = warehouse_Data.columns.str.upper()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.upper()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.upper()



# Remove leading and trailing whitespaces from all columns where dtype is 'O'

warehouse_Data = warehouse_Data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_nb = assest_initiation_invent_nb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_sb = assest_initiation_invent_sb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



# strip the spaces in column 

warehouse_Data.columns = warehouse_Data.columns.str.strip()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.strip()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.strip()


In [12]:
warehouse_Data.count()
# SHAPE OF DATA

print(f"DATASET SHAPE : {warehouse_Data.shape[0]}")

S.R                  3994
METER NO.            3994
BOX NUMBER           3994
PHASE                3994
IME                  3994
FINAL SIM            3994
IMSI                 3994
IP                   3994
WAREHOUSE_UTILITY    3994
dtype: int64

In [13]:
warehouse_Data.columns

Index(['S.R', 'METER NO.', 'BOX NUMBER', 'PHASE', 'IME', 'FINAL SIM', 'IMSI',
       'IP', 'WAREHOUSE_UTILITY'],
      dtype='object')

In [14]:
# concat the assest_initiation_invent_sb and assest_initiation_invent_nb

NB_SB_COMBINED_DATAFRAME = pd.concat([assest_initiation_invent_sb, assest_initiation_invent_nb], ignore_index=True)

In [15]:
# checking is null

NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME["NETWORK_ID"].isnull()].count()

METERNUMBER                                  0
MFG_SERIAL_NUM                               0
UDC_ID                                       0
NETWORK_ID                                   0
INSTALLATIONNUMBER                           0
STATUS                                       0
INSERT_TIME                                  0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    0
dtype: int64

In [16]:
# checking duplicate value

NB_SB_COMBINED_DATAFRAME.duplicated().sum()

0

In [17]:
# droping not used column in the NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop(['METERNUMBER', 'INSTALLATIONNUMBER','INSERT_TIME'], axis=1, inplace=True)

In [20]:
# dop duplicate in  NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop_duplicates(subset=['MFG_SERIAL_NUM','UDC_ID',"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], keep='first', inplace=True, ignore_index=True)

In [21]:
print(f" SHAPE OF NB_SB_COMBINED_DATAFRAME :- {NB_SB_COMBINED_DATAFRAME.shape()[0]}")

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_22524\1323936542.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  NB_SB_COMBINED_DATAFRAME.count()[0]


1707688

# CHANGES IN  NB_SB_COMBINED_DATAFRAME :- DATATYPE,UPPER,SPACES

In [22]:


# Convert all columns to string data type

NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.astype(str)


NB_SB_COMBINED_DATAFRAME.columns = NB_SB_COMBINED_DATAFRAME.columns.str.upper()



# Remove leading and trailing whitespaces from all columns where dtype is 'O'
NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)


# JOIN



In [24]:
#if ((warehouse_Data['METER NO.']== NB_SB_COMBINED_DATAFRAME['MFG_SERIAL_NUM']) and (warehouse_Data['WAREHOUSE_UTILITY']== NB_SB_COMBINED_DATAFRAME["DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"])):
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'WAREHOUSE_UTILITY'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['MFG_SERIAL_NUM'].isnull()]
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_DROPED_DATA = (LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.dropna(subset=['MFG_SERIAL_NUM','UDC_ID']))

if LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL.shape[0] > 0:
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL[warehouse_Data.columns]
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK=pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.'], right_on=['MFG_SERIAL_NUM'], how='left')
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.concat([LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_DROPED_DATA, LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_NULL_CHECK], ignore_index=True)
else:
    LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA


    


In [25]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['MFG_SERIAL_NUM'].isnull()]

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
2140,21,GP4465021,BOX NO - 1161,1 Phase,863709028703454,8991759065212016021,404751250016021,fd00:0000:0800:FB4:0000:0000:0000:0001,SBPDCL,NaN,NaN,NaN,NaN,NaN


In [26]:
#NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME['MFG_SERIAL_NUM']=='GP4465021']

In [27]:


#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'WAREHOUSE_UTILITY'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')


In [28]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.count()

S.R                                          3994
METER NO.                                    3994
BOX NUMBER                                   3994
PHASE                                        3994
IME                                          3994
FINAL SIM                                    3994
IMSI                                         3994
IP                                           3994
WAREHOUSE_UTILITY                            3994
MFG_SERIAL_NUM                               3993
UDC_ID                                       3993
NETWORK_ID                                   3993
STATUS                                       3993
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3993
dtype: int64

In [29]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.csv',index=False)

In [30]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['IMSI'], right_on=['UDC_ID'], how='left')

In [31]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

S.R                                          3994
METER NO.                                    3994
BOX NUMBER                                   3994
PHASE                                        3994
IME                                          3994
FINAL SIM                                    3994
IMSI                                         3994
IP                                           3994
WAREHOUSE_UTILITY                            3994
MFG_SERIAL_NUM                                  0
UDC_ID                                          0
NETWORK_ID                                      0
STATUS                                          0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')       0
dtype: int64

# duplicate row while we join with imsi 

In [32]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[~LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].isnull()].count()

In [33]:
# duplicated row ..  bcz in asset initiation file there are 2 installtion no.  with respect to 1 imsi no.

duplicate_check = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].duplicated(keep=False)]

duplicate_check.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv',index=False)
duplicate_check.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv",index=False)

In [34]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()


S.R                                          3994
METER NO.                                    3994
BOX NUMBER                                   3994
PHASE                                        3994
IME                                          3994
FINAL SIM                                    3994
IMSI                                         3994
IP                                           3994
WAREHOUSE_UTILITY                            3994
MFG_SERIAL_NUM                                  0
UDC_ID                                          0
NETWORK_ID                                      0
STATUS                                          0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')       0
dtype: int64

In [35]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.drop_duplicates(subset=['IMSI'],inplace=True)


In [36]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

S.R                                          3994
METER NO.                                    3994
BOX NUMBER                                   3994
PHASE                                        3994
IME                                          3994
FINAL SIM                                    3994
IMSI                                         3994
IP                                           3994
WAREHOUSE_UTILITY                            3994
MFG_SERIAL_NUM                                  0
UDC_ID                                          0
NETWORK_ID                                      0
STATUS                                          0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')       0
dtype: int64

In [37]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['METER NO.']=='LT0783643']

##### joining both the table "   LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA   "       and            " LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA "

In [38]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [39]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns={"UDC_ID": "METER_MATCH_UDC_ID", "NETWORK_ID": "METER_MATCH_IP", "STATUS": "METER_MATCH_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"METER_MATCH_UTILITY"},
    inplace=True
)



In [40]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column in modified column

In [41]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN

Index(['S.R', 'METER NO.', 'BOX NUMBER', 'PHASE', 'IME', 'FINAL SIM', 'IMSI',
       'IP', 'WAREHOUSE_UTILITY', 'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID',
       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY'],
      dtype='object')

In [42]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN]

In [43]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head()

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL
1,2,GP5020662,BOX NO - 912,1 Phase,869912030364968,8991759065212016002,404751250016002,fd00:0000:0800:FA1:0000:0000:0000:0001,SBPDCL,GP5020662,404751250683612,fd00:0000:0800:E1B:0000:0000:0000:0001,Meter Shop,SBPDCL
2,3,GP5021456,BOX NO - 912,1 Phase,869912033289931,8991759065212016003,404751250016003,fd00:0000:0800:FA2:0000:0000:0000:0001,SBPDCL,GP5021456,404751250683787,fd00:0000:0800:ECA:0000:0000:0000:0001,Retired,SBPDCL
3,4,GP5020781,BOX NO - 912,1 Phase,869912033129327,8991759065212016004,404751250016004,fd00:0000:0800:FA3:0000:0000:0000:0001,SBPDCL,GP5020781,404751250673050,2001:4490:0803:A3D6:0000:0000:0000:0001,Retired,SBPDCL
4,5,GP5021591,BOX NO - 912,1 Phase,869912031725498,8991759065212016005,404751250016005,fd00:0000:0800:FA4:0000:0000:0000:0001,SBPDCL,GP5021591,404751250673736,2001:4490:0803:A684:0000:0000:0000:0001,Retired,SBPDCL


In [44]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED['METER_MATCH_UDC_ID']=='nan'].count()

S.R                    0
METER NO.              0
BOX NUMBER             0
PHASE                  0
IME                    0
FINAL SIM              0
IMSI                   0
IP                     0
WAREHOUSE_UTILITY      0
MFG_SERIAL_NUM         0
METER_MATCH_UDC_ID     0
METER_MATCH_IP         0
METER_MATCH_STATUS     0
METER_MATCH_UTILITY    0
dtype: int64

In [45]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"METER_MATCH_UDC_ID","METER_MATCH_IP","METER_MATCH_STATUS","METER_MATCH_UTILITY"]]

In [46]:
# LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [47]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns = {"MFG_SERIAL_NUM":"IMSI_MATCHED_METER_NO","NETWORK_ID":"IMSI_MATCHED_IP","STATUS":"IMSI_MATCHED_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"IMSI_MATCHED_UTILITY"},
    inplace=True
)

In [48]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.head(1)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,NaN,NaN,NaN,NaN,NaN


In [49]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN=LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN

Index(['S.R', 'METER NO.', 'BOX NUMBER', 'PHASE', 'IME', 'FINAL SIM', 'IMSI',
       'IP', 'WAREHOUSE_UTILITY', 'IMSI_MATCHED_METER_NO', 'UDC_ID',
       'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY'],
      dtype='object')

In [50]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN]

In [51]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,NaN,NaN,NaN,NaN,NaN


In [52]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"IMSI_MATCHED_METER_NO","IMSI_MATCHED_IP","IMSI_MATCHED_STATUS","IMSI_MATCHED_UTILITY"]]

### WORK ON LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED  and  LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED

In [53]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,NaN,NaN,NaN,NaN,NaN
1,2,GP5020662,BOX NO - 912,1 Phase,869912030364968,8991759065212016002,404751250016002,fd00:0000:0800:FA1:0000:0000:0000:0001,SBPDCL,NaN,NaN,NaN,NaN,NaN
2,3,GP5021456,BOX NO - 912,1 Phase,869912033289931,8991759065212016003,404751250016003,fd00:0000:0800:FA2:0000:0000:0000:0001,SBPDCL,NaN,NaN,NaN,NaN,NaN


In [54]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL
1,2,GP5020662,BOX NO - 912,1 Phase,869912030364968,8991759065212016002,404751250016002,fd00:0000:0800:FA1:0000:0000:0000:0001,SBPDCL,GP5020662,404751250683612,fd00:0000:0800:E1B:0000:0000:0000:0001,Meter Shop,SBPDCL
2,3,GP5021456,BOX NO - 912,1 Phase,869912033289931,8991759065212016003,404751250016003,fd00:0000:0800:FA2:0000:0000:0000:0001,SBPDCL,GP5021456,404751250683787,fd00:0000:0800:ECA:0000:0000:0000:0001,Retired,SBPDCL


In [55]:
#

# data correction

In [56]:
# Convert all columns to string data type

# change into str
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)


# ----------------------------------------------------------------------------------

# change into upper
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()


# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)




In [57]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL


# Merge IMSI  and METER  file

In [58]:
warehouse_meter_imsi_data= pd.DataFrame(columns=[warehouse_Data.columns])

In [59]:
warehouse_meter_imsi_data

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY


In [60]:
#  empty column add to the  warehouse_meter_imsi_data  which has already some empty column of warehouse data

In [61]:
warehouse_meter_imsi_data['METER_MATCH_UDC_ID'] = None
warehouse_meter_imsi_data['METER_MATCH_IP'] = None
warehouse_meter_imsi_data['METER_MATCH_STATUS'] = None
warehouse_meter_imsi_data['METER_MATCH_UTILITY'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_IP'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_STATUS'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_UTILITY'] = None


In [62]:
warehouse_meter_imsi_data

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [63]:
# this column are not same in all the dataframe so that i used the above method

In [64]:
"""
warehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'WAREHOUSE_UTILITY',
       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',
       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',
       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',
       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',
       'IMSI_MATCHED_UTILITY'])
"""

"\nwarehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'WAREHOUSE_UTILITY',\n       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',\n       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',\n       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',\n       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',\n       'IMSI_MATCHED_UTILITY'])\n"

In [65]:
warehouse_meter_imsi_data.head()

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [66]:
warehouse_meter_imsi_data = pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED, LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[['METER NO.', 'IMSI','IMSI_MATCHED_METER_NO','IMSI_MATCHED_IP','IMSI_MATCHED_STATUS','IMSI_MATCHED_UTILITY']], left_on=['METER NO.', 'IMSI'], right_on=['METER NO.', 'IMSI'], how='left')

In [67]:
warehouse_meter_imsi_data.head(2)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL,nan,nan,nan,nan
1,2,GP5020662,BOX NO - 912,1 Phase,869912030364968,8991759065212016002,404751250016002,fd00:0000:0800:FA1:0000:0000:0000:0001,SBPDCL,GP5020662,404751250683612,fd00:0000:0800:E1B:0000:0000:0000:0001,Meter Shop,SBPDCL,nan,nan,nan,nan


In [68]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data = warehouse_meter_imsi_data.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data.columns = warehouse_meter_imsi_data.columns.str.upper()

# ----------------------------------------------------------------------------------


# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data = warehouse_meter_imsi_data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



# SIM SET CHECK STATUS

In [69]:
def fun_sim_status_check(row):
    if (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "SAME SET IN SAME UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN SAME UTILITY"
    elif (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "SAME SET IN DIFFERENT UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    else:
        return None

# Apply the function to create a new column
warehouse_meter_imsi_data['SIM SET CHECK STATUS'] = warehouse_meter_imsi_data.apply(fun_sim_status_check, axis=1)



# TRUE - FALSE COLUMN

In [70]:
def fun_check_true_false__METER_SIM(row):
    if row['IMSI']==row['METER_MATCH_UDC_ID']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_SIM"] =  warehouse_meter_imsi_data.apply(fun_check_true_false__METER_SIM, axis=1)

In [71]:
def fun_check_true_false_METER_ORG(row):
    if row['METER_MATCH_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_ORG, axis=1)

In [72]:
def fun_check_true_false_SIM_ORG(row):
    if row['IMSI_MATCHED_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["SIM_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_SIM_ORG, axis=1)

In [73]:
def fun_check_true_false_METER_IP(row):
    if row['METER_MATCH_IP']==row['IP']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_IP"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_IP, axis=1)

# SIM SET CHECK STATUS RECHECK COLUMN

In [74]:
def check_set_and_utility(row):
    if row['METER_SIM'] == True and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SAME SET IN SAME UTILITY"
    elif row['METER_SIM'] == True and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "SAME SET IN DIFFERENT UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "DIFFERENT SET IN SAME UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    elif row['METER_SIM']== False and row['METER_ORG']== True and row['SIM_ORG']== False and row['METER_IP']== False:
        return "REFURBISHED METER WITH NEW IMSI"
    else:
        return None


warehouse_meter_imsi_data["SIM SET CHECK STATUS RECHECK"] =  warehouse_meter_imsi_data.apply(check_set_and_utility, axis=1)


In [75]:
warehouse_meter_imsi_data.columns

Index(['S.R', 'METER NO.', 'BOX NUMBER', 'PHASE', 'IME', 'FINAL SIM', 'IMSI',
       'IP', 'WAREHOUSE_UTILITY', 'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID',
       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',
       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',
       'IMSI_MATCHED_UTILITY', 'SIM SET CHECK STATUS', 'METER_SIM',
       'METER_ORG', 'SIM_ORG', 'METER_IP', 'SIM SET CHECK STATUS RECHECK'],
      dtype='object')

# Action taken column

In [76]:

def fun_action_taken_case_I(row):

    # case 1
    
    if (row['METER_MATCH_STATUS'] == "Installed")  & (row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "SET IS IN INSTALLED STAGE"
    elif (row['METER_MATCH_STATUS'] != "Installed")  & (row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "IMSI IS IN INSTALLED STAGE"
    elif (row['METER_MATCH_STATUS'] == "Installed")  & (row['IMSI_MATCHED_STATUS'] != "Installed"):
        return "METER IS IN INSTALLED STAGE"
    
    # case 2 


    # set 1 and row['METER_IP'] == True
    elif (row['METER_SIM'] == True) and ((row['METER_ORG']  == True) and (row["SIM_ORG"] == True)) and (row['METER_IP'] == True):
        return "METER-SIM LINK FILE REQUIRED"



    # set 1 and row['METER_IP'] == False 
    elif (row['METER_SIM'] == True) and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) and (row['METER_IP'] == False) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    # set 2  "SAME SET IN DIFFERENT UTILITY"
    elif (row['METER_SIM'] == True) and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"

    #set 3  "DIFFERENT SET IN SAME UTILITY"
    
    elif (row['METER_SIM'] == False) and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
 
    # set 4  "DIFFERENT SET IN DIFFERENT UTILITY"
    elif (row['METER_SIM'] == False) and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    


    # new cases are added in this to handle imsi null
    
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan') and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan') and (row['METER_ORG'] == True ):
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan' ) and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan') and (row['METER_ORG'] == False ):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan') and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan')  and ( row['METER_MATCH_UDC_ID'] =='nan') and ( row['METER_MATCH_IP'] =='nan') and ( row['METER_MATCH_STATUS'] =='nan'):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    
    

    
    else:
        return " NEED TO CHECK THIS FILE"
    
    
    
warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_case_I, axis=1)

In [77]:
warehouse_meter_imsi_data[warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO']!='nan'].count()

S.R                             0
METER NO.                       0
BOX NUMBER                      0
PHASE                           0
IME                             0
FINAL SIM                       0
IMSI                            0
IP                              0
WAREHOUSE_UTILITY               0
MFG_SERIAL_NUM                  0
METER_MATCH_UDC_ID              0
METER_MATCH_IP                  0
METER_MATCH_STATUS              0
METER_MATCH_UTILITY             0
IMSI_MATCHED_METER_NO           0
IMSI_MATCHED_IP                 0
IMSI_MATCHED_STATUS             0
IMSI_MATCHED_UTILITY            0
SIM SET CHECK STATUS            0
METER_SIM                       0
METER_ORG                       0
SIM_ORG                         0
METER_IP                        0
SIM SET CHECK STATUS RECHECK    0
ACTION TAKEN                    0
dtype: int64

In [78]:
warehouse_meter_imsi_data.head(2)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED
1,2,GP5020662,BOX NO - 912,1 Phase,869912030364968,8991759065212016002,404751250016002,fd00:0000:0800:FA1:0000:0000:0000:0001,SBPDCL,GP5020662,404751250683612,fd00:0000:0800:E1B:0000:0000:0000:0001,Meter Shop,SBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED


In [79]:
"""

def fun_action_taken_new_case(row):

    if (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == False) :
        return "METER , SIM AND METER SIM LINK REQUIRED"
        
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ) and (row['METER_MATCH_UDC_ID'].isnull()) &  (row['METER_MATCH_IP'].isnull()) &  (row['METER_MATCH_STATUS'].isnull()):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"


warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)

"""


'\n\ndef fun_action_taken_new_case(row):\n\n    if (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == False) :\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n        \n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ) and (row[\'METER_MATCH_UDC_ID\'].isnull()) &  (row[\'METER_MATCH_IP\'].isnull()) &  (row[\'METER_MATCH_STATUS\'].isnull()):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply

In [80]:
"""

def fun_action_taken_new_case(row):
    if (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == False ):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS'])  and (not row['METER_MATCH_UDC_ID']) and (not row['METER_MATCH_IP']) and (not row['METER_MATCH_STATUS']):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"

warehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)


"""


'\n\ndef fun_action_taken_new_case(row):\n    if (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == False ):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\'])  and (not row[\'METER_MATCH_UDC_ID\']) and (not row[\'METER_MATCH_IP\']) and (not row[\'METER_MATCH_STATUS\']):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)\n\n\n'

# Finally done

In [81]:
warehouse_meter_imsi_data.head(1)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED


In [82]:

# Custom function to update meter numbers
def update_meter_number(meter_number):
    if meter_number.startswith('GP') or meter_number.startswith('GL'):
        return 'GOE' + meter_number
    elif meter_number.startswith('LT'):
        return 'LNT' + meter_number
    else:
        return meter_number

# Apply the custom function to each element in the 'METER NO.' column
warehouse_meter_imsi_data['NEW_METER_NUMBER'] = warehouse_meter_imsi_data['METER NO.'].apply(update_meter_number)




array(['1 Phase'], dtype=object)

# FINAL FILE SAVE AT MULTIPLE LOCATION

In [83]:
# Final file at desktop folder

warehouse_meter_imsi_data.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_Data_meter_sim_final.csv',index=False)


# Final file in code folder

warehouse_meter_imsi_data.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv",index=False)

In [84]:
warehouse_meter_imsi_data_check=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv")

In [85]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()

# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)

# NEED TO CHECK FILE

In [86]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM = warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']==' NEED TO CHECK THIS FILE']

In [87]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv",index=False)
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv',index=False)



In [88]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV = pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv")

In [89]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV.head()

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER


In [90]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'].isnull()]

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER


In [91]:
warehouse_meter_imsi_data_check.head(2)


,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686.0,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED,GOEGP5021489
1,2,GP5020662,BOX NO - 912,1 Phase,869912030364968,8991759065212016002,404751250016002,fd00:0000:0800:FA1:0000:0000:0000:0001,SBPDCL,GP5020662,404751250683612.0,fd00:0000:0800:E1B:0000:0000:0000:0001,Meter Shop,SBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED,GOEGP5020662


In [92]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'] != "NEED TO CHECK THIS FILE"].count()

S.R                             3994
METER NO.                       3994
BOX NUMBER                      3994
PHASE                           3994
IME                             3994
FINAL SIM                       3994
IMSI                            3994
IP                              3994
WAREHOUSE_UTILITY               3994
MFG_SERIAL_NUM                  3994
METER_MATCH_UDC_ID              3994
METER_MATCH_IP                  3994
METER_MATCH_STATUS              3994
METER_MATCH_UTILITY             3994
IMSI_MATCHED_METER_NO           3994
IMSI_MATCHED_IP                 3994
IMSI_MATCHED_STATUS             3994
IMSI_MATCHED_UTILITY            3994
SIM SET CHECK STATUS            3994
METER_SIM                       3994
METER_ORG                       3994
SIM_ORG                         3994
METER_IP                        3994
SIM SET CHECK STATUS RECHECK    3994
ACTION TAKEN                    3994
NEW_METER_NUMBER                3994
dtype: int64

In [93]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE',
       'METER , SIM AND METER-SIM LINK FILE REQUIRED'], dtype=object)

In [94]:
warehouse_meter_imsi_data_check.head(2)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER
0,1,GP5021489,BOX NO - 912,1 Phase,869912032074375,8991759065212016001,404751250016001,fd00:0000:0800:FA0:0000:0000:0000:0001,SBPDCL,GP5021489,404751250687686.0,fd00:0000:0800:1E05:0000:0000:0000:0001,Retired,SBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED,GOEGP5021489
1,2,GP5020662,BOX NO - 912,1 Phase,869912030364968,8991759065212016002,404751250016002,fd00:0000:0800:FA1:0000:0000:0000:0001,SBPDCL,GP5020662,404751250683612.0,fd00:0000:0800:E1B:0000:0000:0000:0001,Meter Shop,SBPDCL,nan,nan,nan,nan,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,SIM AND METER-SIM LINK FILE REQUIRED,GOEGP5020662


In [95]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE',
       'METER , SIM AND METER-SIM LINK FILE REQUIRED'], dtype=object)

In [96]:
warehouse_meter_imsi_data_check['SIM SET CHECK STATUS RECHECK'].unique()

array(['REFURBISHED METER WITH NEW IMSI',
       'DIFFERENT SET IN DIFFERENT UTILITY'], dtype=object)

In [97]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['IMSI_MATCHED_METER_NO']=="nan"].count()

S.R                             3994
METER NO.                       3994
BOX NUMBER                      3994
PHASE                           3994
IME                             3994
FINAL SIM                       3994
IMSI                            3994
IP                              3994
WAREHOUSE_UTILITY               3994
MFG_SERIAL_NUM                  3994
METER_MATCH_UDC_ID              3994
METER_MATCH_IP                  3994
METER_MATCH_STATUS              3994
METER_MATCH_UTILITY             3994
IMSI_MATCHED_METER_NO           3994
IMSI_MATCHED_IP                 3994
IMSI_MATCHED_STATUS             3994
IMSI_MATCHED_UTILITY            3994
SIM SET CHECK STATUS            3994
METER_SIM                       3994
METER_ORG                       3994
SIM_ORG                         3994
METER_IP                        3994
SIM SET CHECK STATUS RECHECK    3994
ACTION TAKEN                    3994
NEW_METER_NUMBER                3994
dtype: int64

In [98]:
warehouse_meter_imsi_data_check['METER_MATCH_UDC_ID'].dtype

dtype('O')

In [99]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE',
       'METER , SIM AND METER-SIM LINK FILE REQUIRED'], dtype=object)

In [100]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE'].count()

S.R                             0
METER NO.                       0
BOX NUMBER                      0
PHASE                           0
IME                             0
FINAL SIM                       0
IMSI                            0
IP                              0
WAREHOUSE_UTILITY               0
MFG_SERIAL_NUM                  0
METER_MATCH_UDC_ID              0
METER_MATCH_IP                  0
METER_MATCH_STATUS              0
METER_MATCH_UTILITY             0
IMSI_MATCHED_METER_NO           0
IMSI_MATCHED_IP                 0
IMSI_MATCHED_STATUS             0
IMSI_MATCHED_UTILITY            0
SIM SET CHECK STATUS            0
METER_SIM                       0
METER_ORG                       0
SIM_ORG                         0
METER_IP                        0
SIM SET CHECK STATUS RECHECK    0
ACTION TAKEN                    0
NEW_METER_NUMBER                0
dtype: int64

In [101]:
#assest_initiation_invent_sb[assest_initiation_invent_sb['UDC_ID']=='404751250008001']

In [102]:
#assest_initiation_invent_nb[assest_initiation_invent_nb['UDC_ID']=='404751250008001']

# creating meter-sim link file

In [103]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)
# ----------------------------------------------------------------------------------
# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()
# ----------------------------------------------------------------------------------
# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



In [104]:
#METER , SIM AND METER-SIM LINK FILE REQUIRED

#SIM AND METER-SIM LINK FILE REQUIRED

#METER-SIM LINK FILE REQUIRED


In [105]:
# creating meter_sim_link_file

#CREATE_METER_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER,SIM AND METER-SIM LINK FILE REQUIRED') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER IS IN INSTALLED STAGE') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER , SIM AND METER-SIM LINK FILE REQUIRED')]

CREATE_SIM_AND_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='SIM AND METER-SIM LINK FILE REQUIRED')]

CREATE_METER_SIM_LINK_FILE= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER-SIM LINK FILE REQUIRED')]
                                                                          

In [106]:
print(f" CREATE_METER_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : {CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")
print(f" CREATE_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : {CREATE_SIM_AND_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")
print(f" CREATE_METER_SIM_LINK_FILE - UNIQUE : {CREATE_METER_SIM_LINK_FILE['ACTION TAKEN'].unique()}")


 CREATE_METER_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : ['METER , SIM AND METER-SIM LINK FILE REQUIRED']
 CREATE_SIM_AND_METER_SIM_LINK_FILE - UNIQUE : ['SIM AND METER-SIM LINK FILE REQUIRED']
 CREATE_METER_SIM_LINK_FILE - UNIQUE : []


### Meter-sim link file for all 3 cases

In [107]:
# this file is meter-sim link code

In [108]:
def fun_meter_sim_link_file(row):

    return "{};EP2PMeter;{};{};{}".format(row['NEW_METER_NUMBER'], row['IMSI'],row['FINAL SIM'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
CREATE_METER_SIM_LINK_FILE['meter_sim_link_(M-S)_single_file_data'] = CREATE_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_22524\2655918992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_meter_sim_link_file, axis=1)
C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_22524\2655918992.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND

In [109]:
#EESL_BSNL_PO0000000402_00000_SIM_20231220_121002_SBPDCL
#404751250004001;8991759065212004001;fd00:0000:0800:CC19:0000:0000:0000:0001;;BSNLEDF;ACTIVE;SBPDCL

#  this code is for sim file for all the 2 cases

In [110]:
def fun_sim_file(row):

    return "{};{};{};;BSNLEDF;ACTIVE;{}".format(row['IMSI'], row['FINAL SIM'],row['IP'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)
CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)


C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_22524\669809455.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.apply(fun_sim_file, axis=1)
C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_22524\669809455.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data'] = CREATE_SIM_AND_METER_SIM_LINK_FILE.appl

# THIS CODE IS FOR METER FILE FOR METER,SIM,METER-SIM LINK FILE

In [152]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE'].unique()

array(['1 Phase'], dtype=object)

##### 1 phase and 3 phase

In [143]:
# 1 phase and 3 phase meter file :   meter, sim , meter-sim link file 


CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE']=='1 Phase']
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE['NEW_METER_NUMBER'].str.startswith('GOE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE['NEW_METER_NUMBER'].str.startswith('LNT')]



CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['PHASE']=='3 Phase']
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE['NEW_METER_NUMBER'].str.startswith('GOE')]
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE[CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE['NEW_METER_NUMBER'].str.startswith('LNT')]


In [153]:
CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.head(1)

,S.R,METER NO.,BOX NUMBER,PHASE,IME,FINAL SIM,IMSI,IP,WAREHOUSE_UTILITY,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN,NEW_METER_NUMBER,"meter_sim_link_(M,S & M-S)_single_file_data","sim_link_(M,S & M-S)_single_file_data"
2138,19,GP4465996,BOX NO - 1161,1 Phase,863709026895963,8991759065212016019,404751250016019,fd00:0000:0800:FB2:0000:0000:0000:0001,SBPDCL,GP4465996,404751250685751.0,fd00:0000:0800:1676:0000:0000:0000:0001,Retired,NBPDCL,nan,nan,nan,nan,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER , SIM AND METER-SIM LINK FILE REQUIRED",GOEGP4465996,GOEGP4465996;EP2PMeter;404751250016019;8991759...,404751250016019;8991759065212016019;fd00:0000:...


### 1 PHASE

In [ ]:
def fun_meter_1_PHASE_GENUS_file(row):

    return "{};{};EP2PMeter;6_D1;SKM145-s2;GOE;2019;2019;DLMS;;;G36B7.145090;;;3G;;;;;{};4059;TCP;32;;;;1A2B3C4D;0;1;0;GOE_1PH_WC;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_GENUS.apply(fun_meter_1_PHASE_GENUS_file, axis=1)

In [ ]:
def fun_meter_1_PHASE_LNT_file(row):

    return "{};{};EP2PMeter;6_D1;AURORA;GOE;2021;2021;DLMS;;;RN13.00 00;;8;3G;;;;;{};4059;TCP;32;;;;lnt1;2;1;0;EESL_1PH_WC_V1;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_1_PHASE_LNT.apply(fun_meter_1_PHASE_LNT_file, axis=1)

### 3 PHASE

In [ ]:
def fun_meter_3_PHASE_GENUS_file(row):

    return "{};{};EP2PMeter;7_D2;SKM345-s2;GOE;2019;2019;DLMS;;;GF6B7.325061;;;3G;;;;;{};4059;TCP;32;;;;1A2B3C4D;0;1;0;GOE_3PH_WC;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_GENUS.apply(fun_meter_3_PHASE_GENUS_file, axis=1)

In [ ]:
def fun_meter_3_PHASE_LNT_file(row):

    return "{};{};EP2PMeter;7_D2;REGOR;LNT;2021;2021;DLMS;;;3PBNC.82 01;;8;3G;;;;;{};4059;TCP;32;;;;lnt1;2;1;0;EESL_3PH_WC_V1;1.0;{};{}".format(row['NEW_METER_NUMBER'], row['METER NO.'],row['IME'],row['NEW_METER_NUMBER'],row['WAREHOUSE_UTILITY'])

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT['Meter_link_(M,S & M-S)_single_file_data'] = CREATE_METER_SIM_AND_METER_SIM_LINK_FILE_3_PHASE_LNT.apply(fun_meter_3_PHASE_LNT_file, axis=1)

In [111]:
# SAVING METER-SIM LINK FILE TO OUTPUT LOCATION

CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_METER_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)
CREATE_SIM_AND_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_SIM_AND_METER_SIM_LINK_FILE.csv",index=False)
CREATE_METER_SIM_LINK_FILE.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\CREATE_METER_SIM_LINK_FILE.csv",index=False)

# CREATING A NEW DATA FRAME

#### WORKING ON METER-SIM LINK FILE 

In [112]:
# 1 
# sim-meter link in meter,sim and meter-sim link file

meter_sim_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(M,S & M-S)_single_file_data']
if  not (meter_sim_link_M_S_AND_MS_LINK_file_df.empty):
    meter_sim_link_M_S_AND_MS_LINK_file_df.to_csv("S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_M_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_M_S_AND_MS_LINK.head(1)
else:
    print(" meter_sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


In [113]:
# 2
# sim-meter link in sim and meter-sim link file

meter_sim_link_S_AND_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_S_AND_MS_LINK_file_df=CREATE_SIM_AND_METER_SIM_LINK_FILE['meter_sim_link_(S & M-S)_single_file_data']
if  not (meter_sim_link_S_AND_MS_LINK_file_df.empty):
    meter_sim_link_S_AND_MS_LINK_file_df.to_csv("S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_S_AND_MS_LINK.head(1)
else:
    print(" meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


In [114]:
# 3
# sim-meter link in meter-sim link file

meter_sim_link_MS_LINK_file_df = pd.DataFrame()
meter_sim_link_MS_LINK_file_df=CREATE_METER_SIM_LINK_FILE['meter_sim_link_(M-S)_single_file_data']
if  not (meter_sim_link_MS_LINK_file_df.empty):
    meter_sim_link_MS_LINK_file_df.to_csv("M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)
    EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_MS_LINK.head(1)
else:
    print(" meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


 meter_sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME


### working on sim link file

In [115]:
# 1
# sim link file in meter,sim and meter-sim link file

sim_link_M_S_AND_MS_LINK_file_df = pd.DataFrame()
sim_link_M_S_AND_MS_LINK_file_df=CREATE_METER_SIM_AND_METER_SIM_LINK_FILE['sim_link_(M,S & M-S)_single_file_data'] 
if  not (sim_link_M_S_AND_MS_LINK_file_df.empty):
    sim_link_M_S_AND_MS_LINK_file_df.to_csv("M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None,index=False)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_M_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_M_AND_MS_LINK.head(1)
else:
    print(" sim_link_M_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")


In [116]:
# 2 
# sim link file in sim and meter-sim link file

sim_link_S_AND_MS_LINK_file_df = pd.DataFrame()
sim_link_S_AND_MS_LINK_file_df=CREATE_SIM_AND_METER_SIM_LINK_FILE['sim_link_(S & M-S)_single_file_data']
if  not (sim_link_S_AND_MS_LINK_file_df.empty):
    sim_link_S_AND_MS_LINK_file_df.to_csv("S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None,index=False)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_AND_MS_LINK=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv",header=None)
    EESL_BSNL_PO0000000403_00000_SIM_Link_S_AND_MS_LINK.head(1)
else:
    print(" sim_link_S_AND_MS_LINK_file_df :  EMPTY DATA FRAME")

In [117]:
new_df_check = EESL_GENUS_PO0000000403_00000_SIM_Meter_Link_M_S_AND_MS_LINK.loc[2]
#print(f"len of new_df_check: {new_df_check}")
a=';'
word=""
for i in new_df_check:
    print(i)
    for j in i:
        word=word+j
        if a in j:
            #print(a)
            #print(f"word is : {word}")
            word=""
            #print("semicolon is here")
print(f"last word is : {word}")


GOEGP4465021;EP2PMeter;404751250016021;8991759065212016021;SBPDCL
last word is : SBPDCL


### funciton for del and create the  folder on particular path


In [118]:
def folder_del_create(file_path):
    if os.path.exists(file_path):
        try:
            # Delete the folder and its contents
            shutil.rmtree(file_path)
            os.makedirs(file_path)
            print( "Folder at  deleted successfully.")
            print( "Folder at  created successfully.")
        except Exception as e:
            print(f"Error deleting folder: {e}")


#  METER,SIM AND METER-SIM LINK
M_S_AND_MS_MSLINK_FILE = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'
M_S_AND_MS_M_FILE = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER_FILE'
M_S_AND_MS_S_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\SIM_FILE'

#
S_AND_MSLINK_MSLINK_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE'
S_AND_MSLINK_S_FILE =r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\SIM_FILE'

#METER-SIM LINK
MS_AND_MS_MSLINK_FILE=r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER-SIM LINK\METER-SIM_LINK_FILE'

folder_del_create(M_S_AND_MS_MSLINK_FILE)

Folder at  deleted successfully.
Folder at  created successfully.


In [119]:
folder_del_create(M_S_AND_MS_MSLINK_FILE)
folder_del_create(M_S_AND_MS_M_FILE)
folder_del_create(M_S_AND_MS_S_FILE)

folder_del_create(S_AND_MSLINK_MSLINK_FILE)
folder_del_create(S_AND_MSLINK_S_FILE)


folder_del_create(MS_AND_MS_MSLINK_FILE)

Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.
Folder at  deleted successfully.
Folder at  created successfully.


# function for generate single file

In [120]:
def generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix):
    data_frame_concat_data = pd.read_csv(df_file_path, header=None)

    # base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'

    num_folder = 0
    k = 100
    # new_folder_name = "EESL_GENUS_SINGLE_METER"

    new_folder_path = os.path.join(base_path, new_folder_name)
    os.makedirs(new_folder_path)

    # CURRENT_TIME_STAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Iterate through each row and save it as a separate file
    for index, row in data_frame_concat_data.iterrows():
        
        # Create a new DataFrame with the current row
        single_row_df = pd.DataFrame([row])
        #print(f"single_row_data : {single_row_df}")
        last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
        #print(f"last_word : {last_part_after_semicolon}")


        CURRENT_TIME_STAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
        #print(CURRENT_TIME_STAMP)
        
        #print(k)
        #print("-----------------")
        
        # Create the full path for the new folder

        if k > 995:
            k = 100
            final_new_folder_name = new_folder_name
            num_folder = num_folder + 1
            final_new_folder_name = new_folder_name + "_" + str(num_folder)
            new_folder_path = os.path.join(base_path, final_new_folder_name)
            os.makedirs(new_folder_path)
        else:
            file_name = generated_file_name_prefix + str(k) + generated_file_name_suffix + str(CURRENT_TIME_STAMP) + "_" + str(last_part_after_semicolon) + ".csv"
            #print(f"file name is {file_name}")
            file_path = os.path.join(new_folder_path, file_name)
            #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
            single_row_df.to_csv(file_path, index=False, header=False)
            k=k+1

        
    print("METER-SIM LINK FILE DONE")


In [121]:

# SIM-Meter link file for all the three action taken...

# METER,SIM AND METER-SIM LINK  ----  METER-SIM_LINK_FILE
df_file_path =r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_M_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [122]:

#change the df file path
#  SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE


df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_&_M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [123]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_LINK_EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER-SIM LINK\METER-SIM_LINK_FILE'
new_folder_name = 'EESL_GENUS_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [124]:
# function for sim file 

In [125]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\M_S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\METER,SIM AND METER-SIM LINK\SIM_FILE'
new_folder_name = 'EESL_BSNL_SINGLE_METER'
generated_file_name_prefix = 'EESL_BSNL_PO0000000'
generated_file_name_suffix = '_00000_SIM_Meter_Link_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [126]:


#change the df file path

# METER-SIM LINK\METER-SIM_LINK_FILE

df_file_path = r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\S_AND_MS_LINK_EESL_BSNL_PO0000000403_00000_SIM_Link.csv"
base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\SINGLE_ROW_DATA_OUTPUT\SIM AND METER-SIM LINK FILE\SIM_FILE'
new_folder_name = 'EESL_BSNL_SINGLE_METER'
generated_file_name_prefix = 'EESL_GENUS_PO0000000'
generated_file_name_suffix = '_00000_SIM_'

generate_sim_meter_link_each_row_file(df_file_path,base_path,new_folder_name, generated_file_name_prefix, generated_file_name_suffix)

METER-SIM LINK FILE DONE


In [127]:
#EESL_BSNL_PO0000000402_00000_SIM_20231220_121002_SBPDCL
#404751250004001;8991759065212004001;fd00:0000:0800:CC19:0000:0000:0000:0001;;BSNLEDF;ACTIVE;SBPDCL

In [128]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\METER_IMPORTATION_ZIMP_FILE.zip'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")


Zip file created at: C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\METER_IMPORTATION_ZIMP_FILE.zip


In [129]:
  THE END LETS CHECK

SyntaxError: invalid syntax (4184175093.py, line 1)

# create multiple single file

In [130]:
import pandas as pd

import os,sys
from datetime import datetime

k=100
# Load your file into a pandas DataFrame
data_frame_concat_data = pd.read_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\EESL_GENUS_PO0000000301_00000_SIM_Meter_Link_20240108_113000_SBPDCL.csv', header=None)  # Replace 'your_input_file.csv' with your actual file name

base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

num_folder = 0
new_folder_name = "EESL_GENUS_SINGLE_METER"

new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)

def get_current_time_stamp():
    return f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

CURRENT_TIME_STAMP = get_current_time_stamp()

# Iterate through each row and save it as a separate file
for index, row in data_frame_concat_data.iterrows():
    
    # Create a new DataFrame with the current row
    single_row_df = pd.DataFrame([row])
    #print(f"single_row_data : {single_row_df}")
    last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
    #print(f"last_word : {last_part_after_semicolon}")


    CURRENT_TIME_STAMP = get_current_time_stamp()
    #print(CURRENT_TIME_STAMP)
    
    print(k)
    print("-----------------")
    
    # Create the full path for the new folder

    if k > 995:
        k = 100
        num_folder = num_folder + 1
        new_folder_name = "EESL_GENUS_SINGLE_METER_" + str(num_folder)
        new_folder_path = os.path.join(base_path, new_folder_name)
        os.makedirs(new_folder_path)
    else:
        file_name = f'EESL_GENUS_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_{last_part_after_semicolon}.csv'
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        single_row_df.to_csv(file_path, index=False, header=False)
        k=k+1

    
print("METER-SIM LINK FILE DONE")

100
-----------------
101
-----------------
102
-----------------
103
-----------------
104
-----------------
105
-----------------
106
-----------------
107
-----------------
108
-----------------
109
-----------------
110
-----------------
111
-----------------
112
-----------------
113
-----------------
114
-----------------
115
-----------------
116
-----------------
117
-----------------
118
-----------------
119
-----------------
120
-----------------
121
-----------------
122
-----------------
123
-----------------
124
-----------------
125
-----------------
126
-----------------
127
-----------------
128
-----------------
129
-----------------
130
-----------------
131
-----------------
132
-----------------
133
-----------------
134
-----------------
135
-----------------
136
-----------------
137
-----------------
138
-----------------
139
-----------------
140
-----------------
141
-----------------
142
-----------------
143
-----------------
144
-----------------
145
------

# change into zip file
###### C:\Users\Ratan Kumar Jha\Desktop\EESL_GENUS_PO0000000\data_insert_single_file

In [131]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")


Zip file created at: C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file


# sim link file

In [ ]:
import pandas as pd

import os,sys
from datetime import datetime

k=100
# Load your file into a pandas DataFrame
data_frame_concat_data = pd.read_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\EESL_GENUS_PO0000000100_00000_SIM_20240104_104844_NBPDCL.csv', header=None)  # Replace 'your_input_file.csv' with your actual file name

base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

num_folder = 0
new_folder_name = "EESL_BSNL_SINGLE_METER"

new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)

def get_current_time_stamp():
    return f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

CURRENT_TIME_STAMP = get_current_time_stamp()

# Iterate through each row and save it as a separate file
for index, row in data_frame_concat_data.iterrows():
    
    # Create a new DataFrame with the current row
    single_row_df = pd.DataFrame([row])
    #print(f"single_row_data : {single_row_df}")
    last_part_after_semicolon = row[0].rsplit(';', 1)[-1]
    #print(f"last_word : {last_part_after_semicolon}")


    CURRENT_TIME_STAMP = get_current_time_stamp()
    #print(CURRENT_TIME_STAMP)
    
    print(k)
    print("-----------------")
    
    # Create the full path for the new folder

    if k > 995:
        k = 100
        num_folder = num_folder + 1
        new_folder_name = "EESL_BSNL_SINGLE_METER_" + str(num_folder)
        new_folder_path = os.path.join(base_path, new_folder_name)
        os.makedirs(new_folder_path)
    else:
        file_name = f'EESL_BSNL_PO0000000{k}_00000_SIM_{CURRENT_TIME_STAMP}_{last_part_after_semicolon}.csv'
        #print(f"file name is {file_name}")
        file_path = os.path.join(new_folder_path, file_name)
        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        single_row_df.to_csv(file_path, index=False, header=False)
        k=k+1

    
print("SIM LINK FILE DONE")

100
-----------------
101
-----------------
102
-----------------
103
-----------------
104
-----------------
105
-----------------
106
-----------------
107
-----------------
108
-----------------
109
-----------------
110
-----------------
111
-----------------
112
-----------------
113
-----------------
114
-----------------
115
-----------------
116
-----------------
117
-----------------
118
-----------------
119
-----------------
120
-----------------
121
-----------------
122
-----------------
123
-----------------
124
-----------------
125
-----------------
126
-----------------
127
-----------------
128
-----------------
129
-----------------
130
-----------------
131
-----------------
132
-----------------
133
-----------------
134
-----------------
135
-----------------
136
-----------------
137
-----------------
138
-----------------
139
-----------------
140
-----------------
141
-----------------
142
-----------------
143
-----------------
144
-----------------
145
------

In [ ]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\single data_folder'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")

Zip file created at: C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\zip_file


#### METER,SIM AND METER-SIM LINK
    - METER FILE PREPARE
    - SIM FILE PREPARE
    - SIM-METER LINK

#### SIM AND METER-SIM LINK FILE
    - SIM FILE
    - METER-SIM LINK

#### METER-SIM LINK
    - METER-SIM LINK

